<a id="go_to_start"></a>
# **Embedded Convolutional Neural Network - Tutorial Prático**

Esta é a parte prática do tutorial de construção de um modelo de colorização de imagens classificadas. Nessa `Notebook`, você será guiado a construir código do modelo **ECNN** e também os códigos necessários para **treinar** o modelo, **testar** sua acurácia e **aplicar** o modelo para colorir novas imagens.

Esse tutorial possuirá pouca teoria, espera-se você já tenha lido a parte teórica nessecessária para entender o que será feito aqui presente em nosso [site](https://rafaeldbo.github.io/project-colorization/context). De qualquer maneira, as sessões necessárias para melhor entendimento de cada etapa desse `Notebook` serão referênciadas para consulta na própria etapa. 

## **Sumário**

1. [Preparando as Imagens](#go_to_load_images)
2. [Construindo o Modelo](#go_to_building_model)

    2.1. [Criando a Estrutura do Modelo](#go_to_model_layers)

    2.2. [Função "forward"](#go_to_forward)
    
    2.3. [Modelo Completo](#go_to_ECNN_model)
    
3. [Construindo a Rotina de Treinamento](#go_to_train_model)
4. [Construindo a Rotina de Testes](#go_to_test_model)
5. [Aplicando o Modelo](#go_to_deploy_model)

## **Instalando Dependências**
 
Antes de começarmos, caso não tenha as bibliotecas necessárias, rode a célula abaixo para instala-las ou instale utilizando o arquivo [requirements.txt](https://github.com/rafaeldbo/project-colorization/blob/main/requirements.txt) presente em nosso repositório. É recomendado o uso de um ambiente virtual. 

In [ ]:
# Só é necessário rodar esse script uma vez para instalar as dependências necessárias
!python -m pip install torch scikit-image matplotlib pandas numpy jupyter ipykernel

Caso possua **GPU** com suporte ao `cuda` e deseja utilizá-la, verifique se ela já está disponivel utilizando o código abaixo. Caso não esteja, acesse esse [site](https://pytorch.org/get-started/locally/) e escolha a versão do `pytorch` com maior compatibilidade com sua GPU e a instale.

In [ ]:
import torch

print(f"versão do torch: {torch.__version__}")
print("cuda disponivel!" if torch.cuda.is_available() else "cuda indisponível :(")

In [ ]:

# Importando as bibliotecas necessárias
import time
import pandas as pd
import matplotlib.pyplot as plt
from os import path, listdir
from tqdm import tqdm

import torch
from torch import nn, Tensor, no_grad, cat, from_numpy, save, load
from torch.nn.functional import relu
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb


<a id="go_to_load_images"></a>
## **Preparando as Imagens**

A teoria referente a essa etapa está disponível na sessão [Entradas e Saídas do Modelo](https://rafaeldbo.github.io/project-colorization/inputs_outputs) do nosso site.

Um ponto importante do modelo que criaremos nesse tutorial é que, após ser treinado por um conjunto de imagens de determinadas dimensões $(largura \times altura)$, ele só poderá ser aplicado em imagens destas mesmas dimensões. Nesse tutorial, utilizaremos as imagens do dataset [Image Colorization Dataset](https://www.kaggle.com/datasets/aayush9753/image-colorization-dataset) que possuem as dimensões $400 \times 400$, dessa forma, todas as imagens recebidas e "geradas" pelo modelo possuirão essas mesmas dimensões. Além disso, como se trata de um modelo que colore imagem categorizadas, também precisaremos de um arquivos com as categorias de cada imagem. Felizmente nosso time já preparou o aquivo [categories.csv](https://drive.google.com/uc?export=download&id=115OMNGbthQ5CFmnvPUlxYZ-_Y1CNxI9b) exatamente com essa informação. Coloque tanto as pastas de imagens do dataset quanto o arquivo de categorias em uma pasta `data`, para melhor organização.

Para preparar as imagens para a utilização pelo nosso modelo precisaremos criar um classe de **dataset** personalizada baseada na classe `Dataset` do `pytorch`. Ela deverá localizar todas a imagens que serão utilizadas pelo modelo e possuir uma função `__getitem__` que  caregará uma imagem de cada vez, fornecendo os dados da imagem e sua categoria. Essa estrutura será importante para que, no futuro, o `DataLoader` seja capaz de utilizar essa classe **dataset** para carregar as imagens dos nossos **Batches** (caso não se lembre do que estamos falando, isso será retomado mais adiante). Para facilitar, já iremos fazer com que ela retorne separadamente o *layer* **L** e os *layers* **AB**.

O código desse dataset personalizado será:

In [ ]:
class ImageDataset(Dataset):

    def __init__(self,
        images_path: str, # Caminho da pasta onde estão as imagens 
        categories_file: str, # Caminho do arquivo csv com as categorias das imagens
        size: int = -1, # Quantidade de imagens a serem carregadas, sendo -1 para todas
    ):
        # criando uma lista com os arquivos das imagens
        self.images_path = images_path
        images = listdir(images_path) # listando os arquivos da pasta
        size = size if size > 0 else len(images)
        self.images_files = images[:size]
        
        # criando um dicionário com as categorias das imagens
        df = pd.read_csv(path.join(categories_file), delimiter=';') # lendo o arquivo csv
        df['category'] = df['category'].fillna(0) # colocando a categoria 0 para as imagens sem categoria, caso existam
        self.categories = df.set_index('image')['category'].to_dict() # criando o dicionário

    # função que retorna o tamanho do dataset
    def __len__(self):
        return len(self.images_files)

    def __getitem__(self, 
        index: int, # Índice da imagem a ser carregada
    ) -> tuple[Tensor, Tensor, int]:
        
            # lendo a imagem
            img_file = self.images_files[index]
            img_path = path.join(self.images_path, img_file)
            img = imread(img_path) 

            # converte a imagem de RGB para LAB
            LAB_img = from_numpy(rgb2lab(img)) 
            LAB_img = LAB_img.permute(2, 0, 1) 

            # separa os layers
            gray_layer = LAB_img[0, :, :].unsqueeze(0) 
            color_layers = LAB_img[1:, :, :] 
            
            # retornando o layer L, os layers AB e a categoria da imagem
            return gray_layer.float(), color_layers.float(), self.categories.get(img_file, 0)

<a id=go_to_building_model> </a>
## **Construindo o Modelo**

A teoria referente a essa etapa está disponível em diversas sessões do site, começando pela sessão [Tipo do Modelo](https://rafaeldbo.github.io/project-colorization/model_type). Nosso modelo consistirá em um **Autoencoder** formado por **redes neurais convolucionais** (**CNNs**) e camadas de *embeddings* utilizando um layout `U-Net`.

<a id=go_to_model_layers></a>
### **Criando a Estrutura do Modelo**

Na parte convolucional da U-net (consulte as sessões [Tipos de Convoluções](https://rafaeldbo.github.io/project-colorization/convolutions) e [U-Net](https://rafaeldbo.github.io/project-colorization/unet) para saber mais), teremos:

- 4 níveis nas etapas de **Encoding** e **Decoding**

| `Nível`   | **Encoder**      | **Decoder**                     |
|-----------|------------------|---------------------------------|
| `Nível 0` | 1 layer $^{(1)}$ | 2 layer &  1 layer $^{(1, 2)}$  |
| `Nível 1` | 32 layers        | 32 layers & 32 layers $^{(2)}$  | 
| `Nível 2` | 64 layers        | 64 layers & 64 layers $^{(2)}$  |
| `Nível 3` | 128 layers       | 128 layer & 128 layers $^{(2)}$ |

**OBS¹.:** esse "1 layer" se refere a *layer* de entrada (camada L da imagem).

**OBS².:** utilizamos o símbolo $\&$ para indicar concatenação dos *layers*.

**OBS³.:** os quantidade de *layers* mostrados na tabela indicam a quantidade de *layers* na saída de nível de convolução. 

- 3 camadas de transição
    - Uma convolucional simples de entrada 128 *layers* e saida 256 *layers*
    - duas convolucionais de dilatação que não alteram as dimensões ou os *layers*

- 1 camada convolucional de saida de entrada 3 *layers* (nível 0 do Decoder) e sáida 2 *layers* (objetivo)

- Utilizaremos uma camada de **normalização em batches** após cada camada convolucional, menos no `Nível 0` do **Decoder** e na camada de saída (consulte o tópico [Normalização em Batches](https://rafaeldbo.github.io/project-colorization/batnorm_actfunc/#normalizacao-em-batches-batch-normalization) para saber mais).

Além disso, para acrescentar as informações da categoria no processo utilizaremos uma camada de `embeddings` (consulte a sessão [Camada de Embeddings](https://rafaeldbo.github.io/project-colorization/embeddings/) para saber mais) que será construída utilizando um vetor de 10 caraterísticas e as 8 categorias que possuímos (as `7` que foram usadas para classificar as imagens mais a categoria `0` para as imagens sem categoria). Essa camada será conectada aos *layers* do `Nível 0` do **Encoder** e o `Nível 3` do **Decoder** por meio da concatenação dos *layers* da "imagem" com os *layers* formados pelo vetor de característiscas da categoria (*embbeding*). Não se preocupe se não entendeu, isso ficará mais claro quando aplicarmos durante a construção da função de `fowarding` do modelo.

``` python
# Embeddings
emb_size = 10
embd = nn.Embedding(8, emb_size)

# Encoder
# Nível 1 (1+emb_size -> 32)
conv1 = nn.Conv2d(1 + emb_size, 32, kernel_size=4, stride=2, padding=1)
conv1_bn = nn.BatchNorm2d(32) 

# Nível 2 (32 -> 64)
conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
conv2_bn = nn.BatchNorm2d(64)

# Nível 3 (64 -> 128)
conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
conv3_bn = nn.BatchNorm2d(128)

# Transição
# Convolução de transição (128 -> 256)
conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
conv4_bn = nn.BatchNorm2d(256)

# Primeira dilatação
dilat1 = nn.Conv2d(256, 256, kernel_size=4, stride=1, padding=3, dilation=2)
dilat1_bn = nn.BatchNorm2d(256)

# Segunda dilatação
dilat2 = nn.Conv2d(256, 256, kernel_size=4, stride=1, padding=3, dilation=2)
dilat2_bn = nn.BatchNorm2d(256)

# Decoder
# Nível 3 (256+emb_size -> 128)
tconv3 = nn.ConvTranspose2d(256 + emb_size, 128, kernel_size=4, stride=2, padding=1)
tconv3_bn = nn.BatchNorm2d(128)

# Nível 2 (128+128 -> 64)
tconv2 = nn.ConvTranspose2d(256, 64, kernel_size=4, stride=2, padding=1)
tconv2_bn = nn.BatchNorm2d(64)

# Nível 1 (64+64 -> 32)
tconv1 = nn.ConvTranspose2d(128, 32, kernel_size=4, stride=2, padding=1)
tconv1_bn = nn.BatchNorm2d(32)

# Nível 0 (32+32 -> 2)
tconv0 = nn.ConvTranspose2d(64, 2, kernel_size=4, stride=2, padding=1)

# Saída (2+1 -> 2)
tconv_out = nn.Conv2d(3, 2, kernel_size=3, stride=1, padding=1)
```

Note que como iremos concatenar as informações do *embedding* com as informações da imagem, a entrada das camadas convolucionais iniciais do **Encoder** e **Decoder** terão um número de *layers* maior que o normal (será somado a tamanho do *embedding*).

<a id=go_to_forward> </a>
## **Função `forward`**

A função `forward` é a função que será chamada quando passarmos uma imagem pelo modelo. Ela é responsável por passar a imagem por todas as camadas do modelo e retornar a imagem colorida (consulte [Forwarding](https://rafaeldbo.github.io/project-colorization/unet/#fowarding) para saber mais).

Antes de implementá-la, precisamos entender como será feita a concatenação das informações do *embedding* com as informações da imagem. Faremos isso em 4 passos:

1. **Embedding da Categoria**: a categoria da imagem será passada por uma camada de *embeddings* que transformará a categoria em um vetor de 10 características. 
2. **Reorganização do Embedding**: reorganizaremos o vetor de características para poder ser aplicado a um **batch** de imagens. Isso será feito adicionando mais uma dimensão ao vetor.
3. **Replicação do Embedding**: o vetor de características será replicado para o tamanho da imagem. Isso será feito para que o vetor de características possa ser concatenado com a imagem.
4. **Concatenação**: por fim, concatenaremos o vetor de características formatado com a imagem.

O código responsável por fazer essas 4 etapas é:

```python
embd_category = embd(category) # obtendo o vetor de características
embd_category = embd_category.view(-1, emb_size, 1, 1) # colocando a dimensão extra
embd_category = embd_category.repeat(1, 1, img.shape[2], img.shape[3]) # replicando o vetor

img_embd = cat((img, embd_category), 1) # concatenando a imagem com o vetor formatado
```

Agora que entendemos como será feita a concatenação, podemos implementar a função `forward` do modelo. Nele, nós utilizaremos o `ReLU` como função de ativação (consulte o tópico [Funções de Ativação](https://rafaeldbo.github.io/project-colorization/batnorm_actfunc/#funcoes-de-ativacao-activation-functions) para saber mais). Os níveis serão construidos da seguinte forma:
- **Encoder e Transição**
    - Aplicação da camada de `convolução`;
    - Aplicação da camada de `normalização em batches`;
    - Aplicação da função de ativação `ReLU`.
- **Decoder**
    - Aplicação da camada de `convolução transposta`;
    - Aplicação da camada de `normalização em batches`;
    - Aplicação da função de ativação `ReLU`;
    - Concatenação com a saída do nível correspondente do **Encoder**.

Obtedno a seguinte estrutura:
``` python
def forward(gray, category):
    
    # Primeiro Embedding
    # Obtendo o vetor de características formatado para a concatenação com a imagem de entrada
    embd_decoder = embd(category)\ 
        .view(-1, emb_size, 1, 1)\
        .repeat(1, 1, gray.shape[2], gray.shape[3])
    
    # Concatenação da imagem de entrada com o vetor de características
    gray_embd_encoder = cat((gray, embd_decoder), 1)

    # Encoder
    gray_conv1 = relu(conv1_bn(conv1(gray_embd_encoder))) # Nível 1
    gray_conv2 = relu(conv2_bn(conv2(gray_conv1))) # Nível 2
    gray_conv3 = relu(conv3_bn(conv3(gray_conv2))) # Nível 3

    # Transição
    gray_conv4 = relu(conv4_bn(conv4(gray_conv3))) # convolução de transição
    gray_dilat1 = relu(dilat1_bn(dilat1(gray_conv4))) # primeira dilatação
    gray_dilat2 = relu(dilat2_bn(dilat2(gray_dilat1))) # segunda dilatação

    # Segundo Embedding
    # Obtendo o vetor de características formatado para a concatenação com de entrada no Decoder
    embd_decoder = embd(category)\
        .view(-1, emb_size, 1, 1)\
        .repeat(1, 1, gray_dilat2.shape[2], gray_dilat2.shape[3])
        
    # Concatenação da imagem de entrada do decoder com o vetor de características
    gray_embd_decoder = cat((gray_dilat2, embd_decoder), 1) 

    # Decoder
    gray_tconv3 = relu(tconv3_bn(tconv3(gray_embd_decoder))) # Nível 3
    gray_tconv3 = cat((gray_tconv3, gray_conv3), 1) # concatenação com a saída do nível 3 do Encoder
    gray_tconv2 = relu(tconv2_bn(tconv2(gray_tconv3))) # Nível 2
    gray_tconv2 = cat((gray_tconv2, gray_conv2), 1) # concatenação com a saída do nível 2 do Encoder
    gray_tconv1 = relu(tconv1_bn(tconv1(gray_tconv2))) # Nível 1
    gray_tconv1 = cat((gray_tconv1, gray_conv1), 1) # concatenação com a saída do nível 1 do Encoder
    gray_tconv0 = relu(tconv0(gray_tconv1)) # Nível 0
    gray_tconv0 = cat((gray_tconv0, gray), 1) # concatenação com a imagem de entrada

    output = tconv_out(gray_tconv0) # gerando as camadas AB de saída
    return output
```

<a id="go_to_ECNN_model"> </a>
### **Modelo Completo**

Agora que temos a estrutura do modelo, podemos construir a classe do modelo. Ela será construída utilizando a classe `nn.Module` do `pytorch` como base, terá a função `forward` que acabamos de construir. Além disso, ela terá uma função `__init__` que inicializará todas as camadas do modelo.

In [ ]:
class ECNN_model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        # Embeddings
        self.emb_size = 10
        self.embd = nn.Embedding(8, self.emb_size)

        # Encoder
        self.conv1 = nn.Conv2d(1 + self.emb_size, 32, kernel_size=4, stride=2, padding=1)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3_bn = nn.BatchNorm2d(128)
        
        # Transition
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.conv4_bn = nn.BatchNorm2d(256)
        self.dilat1 = nn.Conv2d(256, 256, kernel_size=4, stride=1, padding=3, dilation=2)
        self.dilat1_bn = nn.BatchNorm2d(256)
        self.dilat2 = nn.Conv2d(256, 256, kernel_size=4, stride=1, padding=3, dilation=2)
        self.dilat2_bn = nn.BatchNorm2d(256)

        # Decoder
        self.tconv3 = nn.ConvTranspose2d(256 + self.emb_size, 128, kernel_size=4, stride=2, padding=1 )
        self.tconv3_bn = nn.BatchNorm2d(128)
        self.tconv2 = nn.ConvTranspose2d(256, 64, kernel_size=4, stride=2, padding=1)
        self.tconv2_bn = nn.BatchNorm2d(64)
        self.tconv1 = nn.ConvTranspose2d(128, 32, kernel_size=4, stride=2, padding=1)
        self.tconv1_bn = nn.BatchNorm2d(32)
        self.tconv0 = nn.ConvTranspose2d(64, 2, kernel_size=4, stride=2, padding=1)

        self.tconv_out = nn.Conv2d(3, 2, kernel_size=3, stride=1, padding=1)

    def forward(
        self,
        gray: Tensor,
        category: int,
    ) -> Tensor:

        # First Embeeding
        embd_decoder = self.embd(category)\
            .view(-1, self.emb_size, 1, 1)\
            .repeat(1, 1, gray.shape[2], gray.shape[3])
        gray_embd_encoder = cat((gray, embd_decoder), 1)

        # Encoder
        gray_conv1 = relu(self.conv1_bn(self.conv1(gray_embd_encoder)))
        gray_conv2 = relu(self.conv2_bn(self.conv2(gray_conv1)))
        gray_conv3 = relu(self.conv3_bn(self.conv3(gray_conv2)))

        # Transition
        gray_conv4 = relu(self.conv4_bn(self.conv4(gray_conv3)))
        gray_dilat1 = relu(self.dilat1_bn(self.dilat1(gray_conv4)))
        gray_dilat2 = relu(self.dilat2_bn(self.dilat2(gray_dilat1)))

        # Second Embeeding
        embd_decoder = self.embd(category)\
            .view(-1, self.emb_size, 1, 1)\
            .repeat(1, 1, gray_dilat2.shape[2], gray_dilat2.shape[3])
        gray_embd_decoder = cat((gray_dilat2, embd_decoder), 1)

        # Decoder
        gray_tconv3 = relu(self.tconv3_bn(self.tconv3(gray_embd_decoder)))
        gray_tconv3 = cat((gray_tconv3, gray_conv3), 1)
        gray_tconv2 = relu(self.tconv2_bn(self.tconv2(gray_tconv3)))
        gray_tconv2 = cat((gray_tconv2, gray_conv2), 1)
        gray_tconv1 = relu(self.tconv1_bn(self.tconv1(gray_tconv2)))
        gray_tconv1 = cat((gray_tconv1, gray_conv1), 1)
        gray_tconv0 = relu(self.tconv0(gray_tconv1))
        gray_tconv0 = cat((gray_tconv0, gray), 1)

        output = self.tconv_out(gray_tconv0)
        return output

<a id="go_to_train_model"> </a>
## **Construindo a Rotina de Treinamento**

A teoria referente a essa etapa está disponível em diversas sessões do site, começando pela sessão [Otimização dos Parâmetros do Modelo](https://rafaeldbo.github.io/project-colorization/optimization). Para treinar um modelo é preciso definir uma função de perda, que irá mensurar os erros do nosso modelo, e um otimizador, que irá corrigir os parâmetros do modelo com base no erro encontrado (consulte os tópicos [Funções de Perda](https://rafaeldbo.github.io/project-colorization/optimization/#funcao-de-perda) e [Otimizador](https://rafaeldbo.github.io/project-colorization/optimization/#otimizador) para saber mais). A função de perda que usaremos será a `MSE` (Mean Squared Error) e o otimizador será o `Adam`.

Para treinar o modelo, utilizaremos um `DataLoader` que carregará os dados do nosso dataset e os dividirá em **batches**. Cada **batch** será passado pelo modelo e, e em seguida, passará pela *função de perda* para mensurar os erros. O otimizador então corrigirá os parâmetros do modelo com base na função de perda. Esse processo será repetido por um número de **epochs** definido por nós (consulte [Batches e Epochs](https://rafaeldbo.github.io/project-colorization/batches_epochs) para saber mais).

Por fim, salvaremos o modelo treinado para que possamos utilizá-lo posteriormente.

**OBS.:** Nesse tutorial, não utilizaremos **multiprocessamento**, o que agilizaria o processo de treino, pois isso demandaria uma estrutura de código mais robusta, que não é a proposta desse tutorial. Caso queira saber mais sobre **multiprocessamento**, consulte a sessão [Multiprocessamento](https://rafaeldbo.github.io/project-colorization/multiprocessing) do nosso site. O código principal que elaboramos, disponível em nosso [repositório](https://github.com/rafaeldbo/project-colorization/tree/main/code), já possui essa funcionalidade, caso queira consultar. 

Antes de construir a rotina de treinamento, vamos definir alguns parâmetros que serão utilizados:

In [ ]:
n_train_images = 64 # quantidade de imagens a serem usados no treinamento
n_test_images = 100 # quantidade de imagens a serem usadas no teste

batch_size = 32 # quantidade de imagens por batch
epochs = 50 # quantidade de Epochs de treinamento
learning_rate = 0.001 # taxa de aprendizado

device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

actual_dir = path.dirname(path.abspath(__name__))

**OBS.:** Por padrão, tentaremos treinar e testar o modelo utilizando a `cuda` da **GPU** (caso esteja disponível). Caso não queria utilizar a `cuda`, basta fixar a variável `device` em `cpu`.

In [ ]:
dataset = ImageDataset(
     images_path = path.join(actual_dir, "data", "train_color"), # caminho para a pasta com as imagens coloridas
     categories_file = path.join(actual_dir, "data", "categories.csv"), # caminho para o arquivo csv com as categorias das imagens
     size = n_train_images # quantidade de imagens a serem carregadas
)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size, # tamanho dos batches que serão gerados
    shuffle=True # os batches serão formados de maneira aleatória
)

ecnn = ECNN_model().to(device) # iniciando o modelo
criterion = nn.MSELoss() # iniciando a função de perda
optimizer = Adam(ecnn.parameters(), lr=learning_rate) # iniciando o otimizador para os parêmtros do modelo e com o learning rate desejado

running_losses = [] # lista para armazenar as perdas de cada epoch

print(f"Número de Parâmetros Treináveis: {sum(p.numel() for p in ecnn.parameters())}")

# Criando a barra de progresso
total_batches = epochs * len(dataloader)
progress_bar = tqdm(total=total_batches, desc="Progresso do Treinamento", position=0)

start = time.time()
for epoch in range(epochs): # para cada epoch...
    epoch_running_loss = 0
    for i, batch in enumerate(dataloader): # para cada batch...
        gray, color, category = batch # extraindo os dados da batch

        # movendo os dados para o dispositivo de treinamento
        gray = gray.to(device) 
        color = color.to(device)
        category = category.to(device)
    
        optimizer.zero_grad() # zerando os gradientes do otimizador
        outputs = ecnn(gray, category) # passando os dados pelo modelo

        loss = criterion(outputs, color) # calculando a perda
        loss.backward() # derivando a perda
        optimizer.step() # atualizando os parâmetros do modelo com base na perda

        epoch_running_loss += loss.item()
        
        # Atualizando a barra de progresso
        progress_bar.update(1)
        progress_bar.set_postfix(epoch=epoch+1, batch=i+1, loss=round(loss.item()/len(batch), 2))
        
    running_losses.append(epoch_running_loss)
    
progress_bar.close()
print(f"Treinamento terminado!")

# Salvando o modelo
save({
        "epoch": epoch,
        "model_state_dict": ecnn.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "time": time.time() - start,
        "running_losses": running_losses
    }, 
    path.join(actual_dir, "ecnn_model.pt")
)

<a id="go_to_test_model"> </a>
## **Construindo a Rotina de Testes**

O método que utilizaremos para testar o modelo é bem simples, usaremos o **modelo treinado** para colorir um **conjunto de imagens de teste** e, em seguida, calcularemos o `MSE` entre as imagens colorizadas e as imagens originais. Faremos isso uilizando uma rotina parecida com a de treinamento, mas sem a necessidade de um otimizador (consulte [Testando o Modelo](https://rafaeldbo.github.io/project-colorization/testing) para saber mais):

In [ ]:
dataset = ImageDataset(
    images_path = path.join(actual_dir, "data", "test_color"), # caminho para a pasta com as imagens coloridas usadas no teste
    categories_file = path.join(actual_dir, "data", "categories.csv"), # caminho para o arquivo csv com as categorias das imagens
    size = n_test_images # quantidade de imagens a serem usadas no teste
)
dataloader = DataLoader(
    dataset, 
    batch_size=batch_size, # tamanho dos batches que serão gerados
    shuffle=False # note que dessa vez, não precisamos que os batches sejam formados de maneira aleatória
)

# Carregando o modelo treinado
ecnn = ECNN_model().to(device)
ecnn.load_state_dict(
    load(
        path.join(actual_dir, "ecnn_model.pt"), 
        weights_only=True # esse parâmetro impede um warning
    )["model_state_dict"]
)

total_loss = 0  

# Criando a barra de progresso
total_batches = len(dataloader)
progress_bar = tqdm(total=total_batches, desc="Testing Progress", position=0, bar_format='{l_bar}{bar:20}{r_bar}')

for i, batch in enumerate(dataloader):
    gray, color, category = batch

    # Movendo os dados para o dispositivo de testes
    gray = gray.to(device)
    color = color.to(device)
    category = category.to(device)
    
    # Passando os dados pelo modelo
    predicted = ecnn(gray, category)
    
    with no_grad(): # impedindo que o gradientes sejam calulados para otimizar os testes
        criterion = nn.MSELoss() # iniciando a função de perda
        loss = criterion(predicted, color) # calculando a perda
        
        total_loss += loss.item() # somando a perda total

    # Atualizando a barra de progresso
    progress_bar.update(1)
    progress_bar.set_postfix(batch=i+1, loss=loss.item())
progress_bar.close()
        
print("Teste Terminado!")
print(f"Erro Médio no Dataset de Teste: {round(total_loss/total_batches, 2)}") 

<a id="go_to_deploy_model"> </a>
## **Aplicando o Modelo**

Agora que o modelo já foi treinado e testado, podemos utilizá-lo para colorir novas imagens. Vamos criar uma função simples que recebe uma imagem em escala de cinza e a categoria da imagem e retorna a imagem colorida:

In [ ]:
def colorize(image_path: str, category: int) -> Tensor:
    ecnn = ECNN_model()
    ecnn.load_state_dict(
        load(
            path.join(actual_dir, "ecnn_model.pt"), 
            weights_only=True
        )["model_state_dict"]
    )
    img = imread(image_path) 

    # Converte a imagem de RGB para LAB
    LAB_img = from_numpy(rgb2lab(img)) 
    LAB_img = LAB_img.permute(2, 0, 1) 

    # Separa o layer L
    gray_layer = LAB_img[0, :, :].unsqueeze(0) 
    
    # Criando batch falso com os dados para passar pelo modelo
    gray = gray_layer.unsqueeze(0).float()
    category = torch.tensor([category])
    
    with no_grad(): # impedindo que o gradientes sejam calulados
        color_layers = ecnn(gray, category)
    
    predicted_LAB_img = cat((gray, color_layers), 1) # concatenando os layers L e AB
    predicted_LAB_img = predicted_LAB_img.squeeze(0) # removendo o batch falso
    predicted_LAB_img = predicted_LAB_img.permute(1, 2, 0) # reorganizando os layes de (Cor, Largura, Altura) para (Largura, Altura, Cor)    
    predicted_RGB_img = lab2rgb(predicted_LAB_img) # convertendo a imagem de LAB para RGB
    
    return predicted_RGB_img

Com a função pronta, basta testar em uma imagem $400 \times 400$ em preto e branco da sua escolha e ver o resultado. Caso não tenha nenhuma, disponibilizamos essa imagem de exemplo: [exemplo.jpg](https://drive.google.com/uc?export=download&id=1cLzdqNwXpkAB58mIc8GjPdD_unGculeG)

In [ ]:
gray_image = imread("./exemplo.jpg")
colorized_img = colorize("./exemplo.jpg", 0)

fig = plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(gray_image)
plt.title("Imagem em Preto e Branco")
plt.axis('off')


plt.subplot(1, 2, 2)
plt.imshow(colorized_img)
plt.title("Imagem Colorida")
plt.axis('off')

plt.show()